In [1]:
import os
import sys
import numpy as np
import pandas as pd
import os.path

data_path="/Users/abry4213/data/fMRI_classification/"

%load_ext rpy2.ipython

In [2]:
%%R
# Load tidyverse R package
suppressPackageStartupMessages({
    library(tidyverse)
    library(see)
    library(cowplot)
    theme_set(theme_cowplot())
})

In addition: Warning message:
package ‘see’ was built under R version 4.3.3 


In [ ]:
# Load 